In [ ]:
%pip --version

In [ ]:
%pip install -r ./requirements.txt

In [ ]:
%pip install -r ./yolov5/requirements.txt

In [ ]:
!python yolov5/train.py --img 640 --batch 16 --epochs 30 --data config/my_data.yaml --weights pretrained_models/yolov5n.pt --cache ram --device 0 --project trained_models --name cars_yolov5n_s640_b16_e30_gpu

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('datasets/cars/annots.csv')

In [ ]:
df

In [1]:
import numpy
import torch
test_torch = torch.rand(100,1,28,28)
print(test_torch.numpy())

[[[[0.6572532  0.70482594 0.7263029  ... 0.326644   0.28501588
    0.8413595 ]
   [0.7854287  0.28594244 0.46733433 ... 0.12058216 0.05241877
    0.94500804]
   [0.89178175 0.00423378 0.7397486  ... 0.9612487  0.84900385
    0.3521257 ]
   ...
   [0.5473209  0.5183219  0.44464397 ... 0.8330286  0.5884132
    0.87803817]
   [0.90145093 0.01764375 0.06275761 ... 0.10863328 0.24017787
    0.41394138]
   [0.73503166 0.35286248 0.70794857 ... 0.34303194 0.11552674
    0.22115082]]]


 [[[0.85692227 0.8438705  0.33396447 ... 0.52632326 0.16659075
    0.64024055]
   [0.21657473 0.5467942  0.45014453 ... 0.5174344  0.2377758
    0.43943667]
   [0.78972816 0.13960117 0.37889767 ... 0.6572265  0.35805416
    0.14245605]
   ...
   [0.5615749  0.5265852  0.99246895 ... 0.07655376 0.6319934
    0.10621578]
   [0.68255055 0.88951486 0.11953878 ... 0.32422715 0.36445755
    0.205899  ]
   [0.02791131 0.80538166 0.15718347 ... 0.37147206 0.7012571
    0.77953374]]]


 [[[0.9914501  0.46555817 0.316003

/home/darkroom2/.pyenv/versions/3.10.4/envs/yolo/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
grouped_df = df.groupby(['image']).agg({
    'xmin': list,
    'ymin': list,
    'xmax': list,
    'ymax': list
})

In [ ]:
grouped_df

SKRYPT KONWERTUJĄCY CSV NA YOLO

Poniższe ścieżki należy uzupełnić. Dodatkowo plik `classes.names` należy utworzyć zgodnie ze schematem:
```text
class_name_0
class_name_1
class_name_2
class_name_3
class_name_4
class_name_5
```

In [ ]:
# Sciezka do starych anotacji (labeli) w formacie csv
csv_path = 'datasets/cars/annots.csv'
# Sciezka do obrazow
images_path = 'datasets/cars/training_images'
# Sciezka do pliku z nazwami klas (yolo wymaga ID klasy, a nie nazwy!)
classes_names_path = 'datasets/cars/classes.names'
# Sciezka do zapisu datasetu w nowym formacie
dataset_path = 'datasets/cars_converted'

In [ ]:
from collections import defaultdict
from pathlib import Path
from shutil import copy

from PIL import Image

classes_names_file = Path(classes_names_path)
classes_names_list = classes_names_file.read_text(encoding='utf8').split()
class_name_id_map = {class_name: idx for idx, class_name in enumerate(classes_names_list)}

images_old_dir = Path(images_path)
dataset_dir = Path(dataset_path)
images_new_dir = dataset_dir / 'images'
images_new_dir.mkdir(exist_ok=True, parents=True)
labels_dir = dataset_dir / 'labels'
labels_dir.mkdir(exist_ok=True, parents=True)

csv_file = Path(csv_path)
csv_content = csv_file.read_text(encoding='utf8')
csv_lines = csv_content.split()
csv_rows = [line.split(',') for line in csv_lines]
header_row = csv_rows[0]
# Mapowanie nazw kolumn na index (kolejność) na liście z nagłówkiem
col_idx = {col_name: idx for idx, col_name in enumerate(header_row)}
data_rows = csv_rows[1:]

file_bounds_map = defaultdict(list)

for row in data_rows:
    filename = str(row[col_idx['image']])
    class_name = classes_names_list[0]
    xmin = float(row[col_idx['xmin']])
    ymin = float(row[col_idx['ymin']])
    xmax = float(row[col_idx['xmax']])
    ymax = float(row[col_idx['ymax']])

    # Format yolo:
    # class x_center y_center width height

    image_file = images_old_dir / filename
    img_w, img_h = Image.open(image_file).size

    w = xmax - xmin
    h = ymax - ymin
    x_cnt = (xmin + xmax) / 2.0
    y_cnt = (ymin + ymax) / 2.0

    norm_bounds = [x_cnt / img_w, y_cnt / img_h, w / img_w, h / img_h]
    bounds_str = ' '.join(f'{i:.6f}' for i in norm_bounds)

    yolo_fmt_str = f'{class_name_id_map.get(class_name, 0)} {bounds_str}'
    file_bounds_map[filename].append(yolo_fmt_str)

for filename, bounds_list in file_bounds_map.items():
    image_old_file = images_old_dir / filename
    image_new_file = images_new_dir / filename
    copy(image_old_file, image_new_file)

    label_file_name = image_new_file.with_suffix('.txt').name
    label_file = labels_dir.joinpath(label_file_name)
    label_file_content = '\n'.join(bounds_list)
    label_file.write_text(label_file_content, encoding='utf8')

Splitowanie folderów zawierających obrazki oraz labelki na foldery `train` / `val` / `test`.

In [ ]:
images_dir = 'datasets/cars_converted/images'
labels_dir = 'datasets/cars_converted/labels'

seed = 1337
ratio = (.8, .1, .1)

In [ ]:
from shutil import move
import random

for directory_path in [images_dir, labels_dir]:
    random.seed(seed)  # make sure its reproducible
    directory = Path(directory_path)
    files = [f for f in directory.iterdir() if f.is_file() and not f.name.startswith(".")]
    files.sort()
    random.shuffle(files)

    # the data was shuffled already
    split_train_idx = int(round(ratio[0] * len(files)))
    split_val_idx = split_train_idx + int(round(ratio[1] * len(files)))

    use_test = len(ratio) == 3

    files_train = files[:split_train_idx]
    files_val = files[split_train_idx:split_val_idx] if use_test else files[split_train_idx:]

    li = [(files_train, "train"), (files_val, "val")]

    # optional test folder
    if use_test:
        files_test = files[split_val_idx:]
        li.append((files_test, "test"))

    for files, folder_type in li:
        end_path = directory / folder_type
        Path(end_path).mkdir(parents=True, exist_ok=True)
        for f in files:
            move(str(f), str(end_path))

In [ ]:
from shutil import move
import random

for directory_path in [images_dir, labels_dir]:
    random.seed(seed)  # make sure its reproducible
    directory = Path(directory_path)
    files = [f for f in directory.iterdir() if f.is_file() and not f.name.startswith(".")]
    files.sort()
    random.shuffle(files)

    # the data was shuffled already
    split_train_idx = int(round(ratio[0] * len(files)))
    split_val_idx = split_train_idx + int(round(ratio[1] * len(files)))

    use_test = len(ratio) == 3

    files_train = files[:split_train_idx]
    files_val = files[split_train_idx:split_val_idx] if use_test else files[split_train_idx:]

    li = [(files_train, "train"), (files_val, "val")]

    # optional test folder
    if use_test:
        files_test = files[split_val_idx:]
        li.append((files_test, "test"))

    for files, folder_type in li:
        end_path = directory / folder_type
        Path(end_path).mkdir(parents=True, exist_ok=True)
        for f in files:
            move(str(f), str(end_path))